In [23]:
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import torch 

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [26]:
class Layer: 
    def __init__ (Id, Od, p_activ, final = False):
        self.W = (torch.randn(Od, Id) * torch.sqrt(2/Id)).to(device)
        self.dW = torch.zeros_like(self.W).to(device)
        
        self.B, self.dB, self.dB_rms, self.dB_vel, self.C, self.dC, self.dC_rms, 
        self.dC_vel, self.V_avg, self.C_avg, self.dV, self.dV_rms, self.dV_vel = [torch.zeros(Od, 1).to(device) for _ in range (0,13)]
        self.p = p_active
        
        self.V = torch.ones_like(self.C).to(device)
        
        self.cache = {}
        self.dA_prev = None
        self.final = final 
        
        def opt(self, opt_params):
            f.optimize(self, opt_params)
        
        def fp (self, A_prev, Y):
            if self.final:
                f.fin_fprop(self, A_prev, Y)
            else:
                f.fprop(self, A_prev, Y)
        
        def bp (self, dA, Y): 
            if self.final:
                f.fin_bprop(self, dA, Y)
            else:
                f.bprop(self, dA, Y)

In [28]:
class f: 
    @staticmethod
    def prelu (Z,p): 
        return torch.where(Z>0, Z, p * Z)
        
    @staticmethod
    def prelu_grad (Z,p): 
        return torch.where(Z>0, 1, p) 
        
    @staticmethod
    def softmax (Z): 
        Z_exp = torch.exp(Z - torch.max(Z))
        return Z_exp/torch.sum(Z_exp, dim = 0, keepdim = True)
        
    @staticmethod
    def crossENT (A,Y): 
        m = A.shape[1]
        L = -torch.sum(Y * torch.log(A), dim = 0, keepdim = True)
        J = torch.sum(L)/m 
        return J
    
    @staticmethod
    def fprop (layer, A_prev, Y):
        e = 0.0000000000001
        Z = (torch.matmul(layer.W, A_prev) + layer.B).to(device)
        mean = torch.mean(Z, dim = 1, keepdim = True).to(device)
        std = (torch.std(Z, dim = 1, keepdim = True) + e).to(device)
        Z_ = ((Z - mean)/std).to(device)
        H = ((layer.V * Z_) + layer.C).to(device)
        A = (f.prelu(H, layer.p)).to(device)
        layer.cache = {'A_prev': A_prev.to(device),
                       'Z' : Z,
                       'std': std,
                       'H': H,
                       'Z_': Z_,
                       'A' : A
                      }
        
    @staticmethod
    def fin_fprop (layer, A_prev, Y):
        e = 0.0000000000001
        Z = (torch.matmul(layer.W, A_prev) + layer.B).to(device)
        mean = torch.mean(Z, dim = 1, keepdim = True).to(device)
        std = (torch.std(Z, dim = 1, keepdim = True) + e).to(device)
        Z_ = ((Z - mean)/std).to(device)
        H = ((layer.V * Z_) + layer.C).to(device)
        A = (f.softmax(H, layer.p)).to(device)
        layer.cache = {'A_prev': A_prev.to(device),
                       'Z' : Z,
                       'std': std,
                       'H': H,
                       'Z_': Z_,
                       'A' : A
                      }
        self.cost = f.crossENT(A, Y)
        
    @staticmethod
    def bprop (layers, dA, Y):
            A_prev = layer.cache['A_prev'].to(device)
            Z = layer.cache['Z'].to(device)
            Z_ = layer.cache['Z_'].to(device)
            A = layer.cache['A'].to(device)
            m = A_prev.shape[1]
            std = layer.cahse['std'].to(device)
            dH = (dA * f.prelu_grad(H, layer.p)).to(device)
            layer.dV = torch.sum((Z_ * dH), dim = 1, keepdim = True)
            layer.dC = torch.sum(dH, dim = 1, keepdim = True)
            dZ_ = (layer.V * dH).to(device)
            dZ = (((m*dZ_) - (torch.sum(dZ_, dim = 1, keepdim = True)) - (Z_ * (toch.sum((Z_ * dZ_), dim = 1, keepdims = True))))/(m * std)).to(device)
            layer.dW = torch.matmul(dZ, A_prev.permute(1, 0))
            layer.dB = torch.sum(dZ, dim = 1, keepdim = True)
            layer.dA_prev = torch.matmul(W.permute(1, 0), dZ)
                             
    @staticmethod
    def bprop (layers, dA, Y):
            A_prev = layer.cache['A_prev'].to(device)
            Z = layer.cache['Z'].to(device)
            Z_ = layer.cache['Z_'].to(device)
            A = layer.cache['A'].to(device)
            m = A_prev.shape[1]
            std = layer.cahse['std'].to(device)
            dH = ((A - Y)/m).to(device)
            layer.dV = torch.sum((Z_ * dH), dim = 1, keepdim = True)
            layer.dC = torch.sum(dH, dim = 1, keepdim = True)
            dZ_ = (layer.V * dH).to(device)
            dZ = (((m*dZ_) - (torch.sum(dZ_, dim = 1, keepdim = True)) - (Z_ * (toch.sum((Z_ * dZ_), dim = 1, keepdims = True))))/(m * std)).to(device)
            layer.dW = torch.matmul(dZ, A_prev.permute(1, 0))
            layer.dB = torch.sum(dZ, dim = 1, keepdim = True)
            layer.dA_prev = torch.matmul(W.permute(1, 0), dZ)
    
    @staticmethod
    def lr_decay(t, alpha_0):
        alph = alpha_0
        return alph
    
    @staticmethod
    def optimize(layers, opt_params):
        alpha_0 = opt_params['alpha_0']
        t = opt_params['t']
        alpha = f.lr_decay(t, alpha_0)
        beta_dW, beta_dW0 = opt_params['beta_dW'] 
        beta_dB, beta_dB0 = opt_params['beta_dB']
        beta_dC, beta_dC0 = opt_params['beta_dC']
        theta_C = opt_params['theta_C']
        theta_V = opt_params['theta_V']
        
        layer.dB_vel = (((beta_dB) * (layer.dB_vel)) + ((1-Beta_dB) * (layer.dB)))/(1 - (beta_dB ** t))
        layer.dB_rms = (((beta_dB0) * (layer.dB_vel)) + ((1-Beta_dB0) * ((layer.dB) ** 2)))/(1 - (beta_dB0 ** t))
                                                        
        layer.dW_vel = (((beta_dW) * (layer.dW_vel)) + ((1-Beta_dW) * (layer.dW)))/(1 - (beta_dW ** t))
        layer.dW_rms = (((beta_dW0) * (layer.dW_vel)) + ((1-Beta_dW0) * ((layer.dW) ** 2)))/(1 - (beta_dW0 ** t))
                                                        
        layer.dV_vel = (((beta_dV) * (layer.dV_vel)) + ((1-Beta_dV) * (layer.dV)))/(1 - (beta_dV ** t))
        layer.dV_rms = (((beta_dV0) * (layer.dV_vel)) + ((1-Beta_dV0) * ((layer.dV) ** 2)))/(1 - (beta_dV0 ** t))
                                                        
        layer.dC_vel = (((beta_dC) * (layer.dC_vel)) + ((1-Beta_dC) * (layer.dC)))/(1 - (beta_dC ** t))
        layer.dC_rms = (((beta_dC0) * (layer.dC_vel)) + ((1-Beta_dC0) * ((layer.dC) ** 2)))/(1 - (beta_dC0 ** t))
                                                        
        layer.V_avg = (((theta_V) * (layer.V_avg)) + ((1 - theta_V) * (layer.V)))/(1 - (theta_V ** t))
        layer.C_avg = (((theta_C) * (layer.C_avg)) + ((1 - theta_C) * (layer.C)))/(1 - (theta_C ** t))
        
        layer.W -= (alpha) * (layer.dW_vel/torch.sqrt(layer.dW_rms))
        layer.B-= (alpha) * (layer.dB_vel/torch.sqrt(layer.dB_rms))
        layer.V -= (alpha) * (layer.dV_vel/torch.sqrt(layer.dV_rms))
        layer.C -= (alpha) * (layer.dC_vel/torch.sqrt(layer.dC_rms))
        
        

In [29]:
class Model: 
    def __init__ (self, A0_dim, AL_dim, HL_dims, p_activ): 
        self.L_dims = [A0_dim] + HL_dims + [AL_dim]
        self.n = len(self.L_dims)
        self.layers = [0]
        self.layers += [Layer(self.L_dims[i-1], self.L_dims[i], p_activ, final = False) for i in range (1, self.n - 1)]
        self.layers += [Layer(self.L_dims[self.n - 2], self.L_dims[self.n - 1], p_activ, final = True)]

    def forward (self, X, Y):
        A_prev = X
        for i in range(1, self.n): 
            
            self.layers[i].fp(A_prev, Y)
            A_prev = self.layers[i].cache['A']
            #self.fp = F.fprop(self, A_prev, Y) 
    def backward (self, X, Y): 
        dA = None 
        for i in reversed(range(1, self.n)): 
            self.layers[i].bp(dA, Y)
            dA = self.layers[i].dA_prev

    def gradient_descent(self, alph): 
        for i in range(1, self.n): 
            self.layers[i].opt(opt_params)

    def training_epoch (self, opt)params, X_train_batches, Y_train_batches, printCost = False):
        bs = len(X_train_batches)
        for t in range (0,bs): 
            X_train = X_train_batches[t]
            Y_train = Y_train_batches[t]
            
            self.forward(X_train, Y_train)
            self.backward(X_train, Y_train)
            self.gradient_descent(alph)
            if printCost:
                print(self.layers[self.n-1].cost)
        
    def predict (self, X, Y): 
        self.forward(X, Y)
        fin_act = self.layers[self.n - 1].cache['A']
        pred = torch.zeros_like(fin_act)
        max_indices = torch.argmax(fin_act, dim=0)
        pred.scatter_(0, max_indices.unsqueeze(0), 1)
        return pred